In [2]:
# Import pickle
import pickle

import numpy as np

import matplotlib.pyplot as plt

import statsmodels.api as sm

import seaborn as sns

import pylab as py

import scipy.stats as stats

import seaborn as sns

import matplotlib.pyplot as plt

from IPython.display import clear_output

import os

os.chdir('C:\\Users\\HADDAM\\Documents\\Python Scripts\\multi_flow_decomp\\')




##########################  Baseline heuristics handling  ########################## 
"""
{"res_key_metadata":res_key_metadata,
"res_value_metadata":res_value_metadata, 
"data":deepcopy(dict(dict_results))}

res_key_metadata = ["nb_nodes", "nb_pairs", "id_instance",
                    ("path_type_selector", "path_card_criteria", 
                    "lr_rate", "coeffs_list")]

res_value_metadata = ["flow_val_residue", "flow_residue", 
                        "multi_flow_residue", "prop_flow_support", 
                        "prop_shortest_paths", "transition_function_residue",
                        "reward"]
"""

with open("results/simulated/MFDS_vs_RL/results_test/results_node_pairs_rl_heuristics.pickle", 
          'rb') as handle:
    file = pickle.load(handle)
    test_names = file["res_key_metadata"]
    metric_names = file["res_value_metadata"]
    data_res = file["data"]

# Number of instances
nb_instances = len({id_inst for _, _, id_inst, _ in data_res.keys()})

# The learning rates
lr_rates = set(lr_rate for _, _, _, (_, _, lr_rate, _) in data_res)

node_pairs_list = list({(nb_nodes, nb_pairs) for nb_nodes, nb_pairs, _, _ in data_res})
node_pairs_list.sort()

# Calculate the best learning rate for the multiple path case
mean_rew_by_lr_rate = {(nb_nodes, nb_pairs):{np.mean([data_res[(nb_nodes, 
                                                                nb_pairs, 
                                                                ind_instance, 
                                                                test_infos)][-1] 
                                                for (n1, n2, ind_instance, test_infos) in data_res
                                                    if test_infos[2] == lr_rate and\
                                                        n1 == nb_nodes and\
                                                        n2 == nb_pairs]):lr_rate 
                                                            for lr_rate in lr_rates}
                                                                for (nb_nodes, nb_pairs) in node_pairs_list}
lr_rate_max_by_np = {key:mean_rew_by_lr_rate[key][max(mean_rew_by_lr_rate[key].keys())] 
                                                for key in mean_rew_by_lr_rate}


# Construct the data to be displayed
display_data = dict()
for i in range(len(metric_names)):
    name_metric = metric_names[i]
    for (nb_nodes, nb_pairs) in node_pairs_list:
        lr_rate = lr_rate_max_by_np[(nb_nodes, nb_pairs)]
        data_node_pair = [data_res[key[0], key[1], key[2], key[3]][i]
                                for key in data_res
                                    if key[0] == nb_nodes and\
                                            key[1] == nb_pairs and\
                                                key[3][2] == lr_rate]
        mean_ = np.mean(data_node_pair)
        std_ = np.std(data_node_pair)
        display_data[nb_nodes, nb_pairs, name_metric] = (mean_, std_)
print(display_data)
print(node_pairs_list)

"""
Display data structure
{<nb_nodes, nb_pairs, name_metric> : <(mean_, std_)>, ... }
"""

# Create the heatmaps associated to the data
save_path = "results/simulated/MFDS_vs_RL/results_test/node_pairs"
y_labels = list({nb_nodes for nb_nodes, _ in node_pairs_list})
y_labels.sort()
nb_nb_nodes = len(y_labels)
x_labels = list({nb_pairs for _, nb_pairs in node_pairs_list})
x_labels.sort()
nb_nb_pairs = len(x_labels)
process_derivative_names = {"multi_flow_residue"}
heatmaps_data_process_derivative_names = dict()
for i in range(len(metric_names)):
    name_metric = metric_names[i]
    heat_map_data = [display_data[nb_nodes, nb_pairs, name_metric][0] 
                        for nb_nodes, nb_pairs in node_pairs_list]
    print(heat_map_data)
    heat_map_data = [[heat_map_data[i*nb_nb_pairs + j] for j in range(nb_nb_pairs)]
                                                            for i in range(nb_nb_nodes)]
    if name_metric in process_derivative_names:
        heatmaps_data_process_derivative_names[name_metric] = heat_map_data
    colormap = sns.color_palette(palette = "coolwarm", as_cmap = True)
    heat_map = sns.heatmap(heat_map_data, 
                            annot = True, 
                            fmt = ".4f", 
                            cmap = colormap,
                            xticklabels = x_labels, 
                            yticklabels = y_labels)
    plt.xlabel("pairs", fontsize=13)
    plt.ylabel("nodes", fontsize=13)
    if save_path:
        fig = heat_map.get_figure() 
        fig.savefig(os.path.join(save_path, 
                                "node_pairs_rl_"+name_metric), 
                                bbox_inches='tight', 
                                pad_inches=0)
    fig.clear()


# Create the heatmaps associated to the data
for name_metric in heatmaps_data_process_derivative_names:
    # Fetch corresponding 'heat_map_data'
    heat_map_data = heatmaps_data_process_derivative_names[name_metric]

    # process 'pairs' derivative
    heat_map_data_deriv = [[1000*(row[j+1] - row[j])/(x_labels[j+1] - x_labels[j]) 
                                            for j in range(len(row)-1)] 
                                                for row in heat_map_data]
    colormap = sns.color_palette(palette = "coolwarm", as_cmap = True)
    heat_map = sns.heatmap(heat_map_data_deriv, 
                            annot = True, 
                            fmt = ".4f", 
                            cmap = colormap,
                            xticklabels = x_labels[:-1], 
                            yticklabels = y_labels)
    plt.xlabel("pairs", fontsize=13)
    plt.ylabel("nodes", fontsize=13)
    if save_path:
        fig = heat_map.get_figure() 
        fig.savefig(os.path.join(save_path, 
                                "node_pairs_rl_pairs_deriv_"+name_metric), 
                                bbox_inches='tight', 
                                pad_inches=0)
    fig.clear()

    # 'nodes' derivative
    heat_map_data_deriv = [[1000*(heat_map_data[i+1][j] - heat_map_data[i][j])/(y_labels[i+1] - y_labels[i]) 
                                                for j in range(len(heat_map_data[0]))]
                                                    for i in range(len(heat_map_data)-1)]
    colormap = sns.color_palette(palette = "coolwarm", as_cmap = True)
    heat_map = sns.heatmap(heat_map_data_deriv, 
                            annot = True, 
                            fmt = ".4f", 
                            cmap = colormap,
                            xticklabels = x_labels, 
                            yticklabels = y_labels[:-1])
    plt.xlabel("pairs", fontsize=13)
    plt.ylabel("nodes", fontsize=13)
    if save_path:
        fig = heat_map.get_figure() 
        fig.savefig(os.path.join(save_path, 
                                "node_pairs_rl_nodes_deriv_"+name_metric), 
                                bbox_inches='tight', 
                                pad_inches=0)
    fig.clear()

{(55, 10, 'flow_val_residue'): (0.02681730128342727, 0.01790044061374397), (55, 15, 'flow_val_residue'): (0.03713407737342178, 0.017790166759494776), (55, 20, 'flow_val_residue'): (0.051031956752762314, 0.02136584970849533), (75, 10, 'flow_val_residue'): (0.03455387452628067, 0.02073796453266218), (75, 15, 'flow_val_residue'): (0.05265499322217969, 0.02114723574867637), (75, 20, 'flow_val_residue'): (0.06330870199140644, 0.023756729150886335), (95, 10, 'flow_val_residue'): (0.03703824226981925, 0.018516221418966375), (95, 15, 'flow_val_residue'): (0.056612972143332635, 0.020769881017532875), (95, 20, 'flow_val_residue'): (0.07268874240341075, 0.019440360234890236), (55, 10, 'flow_residue'): (0.033226623701263856, 0.020371584359422455), (55, 15, 'flow_residue'): (0.04650963378232813, 0.0201124420515171), (55, 20, 'flow_residue'): (0.06110849095868478, 0.021750416479553284), (75, 10, 'flow_residue'): (0.04071745918234579, 0.0209090684331855), (75, 15, 'flow_residue'): (0.0598674902842931

<Figure size 640x480 with 0 Axes>